# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 10 2023 10:20AM,254643,20,9014380,"Exact-Rx, Inc.",Released
1,Jan 10 2023 10:20AM,254643,20,9014381,"Exact-Rx, Inc.",Released
2,Jan 10 2023 10:20AM,254643,20,9014382,"Exact-Rx, Inc.",Released
3,Jan 10 2023 10:20AM,254643,20,9014383,"Exact-Rx, Inc.",Released
4,Jan 10 2023 10:20AM,254643,20,9014384,"Exact-Rx, Inc.",Released
5,Jan 10 2023 10:20AM,254643,20,9014385,"Exact-Rx, Inc.",Released
6,Jan 10 2023 10:20AM,254643,20,9014386,"Exact-Rx, Inc.",Released
7,Jan 10 2023 10:20AM,254643,20,9014387,"Exact-Rx, Inc.",Released
8,Jan 10 2023 10:20AM,254643,20,9014388,"Exact-Rx, Inc.",Released
9,Jan 10 2023 10:20AM,254643,20,9014389,"Exact-Rx, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
31,254639,Released,1
32,254640,Released,8
33,254641,Released,15
34,254642,Released,4
35,254643,Released,10


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
254639,NaN,NaN,1.0
254640,NaN,NaN,8.0
254641,NaN,NaN,15.0
254642,NaN,NaN,4.0
254643,NaN,NaN,10.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254563,37.0,2.0,1.0
254566,0.0,0.0,36.0
254567,0.0,0.0,10.0
254580,3.0,1.0,0.0
254585,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254563,37,2,1
254566,0,0,36
254567,0,0,10
254580,3,1,0
254585,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254563,37,2,1
1,254566,0,0,36
2,254567,0,0,10
3,254580,3,1,0
4,254585,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254563,37,2,1
1,254566,,,36
2,254567,,,10
3,254580,3,1,
4,254585,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 10 2023 10:20AM,254643,20,"Exact-Rx, Inc."
10,Jan 10 2023 10:18AM,254641,10,"Methapharm, Inc."
25,Jan 10 2023 10:15AM,254632,10,ISDIN Corporation
84,Jan 10 2023 10:14AM,254642,10,"Methapharm, Inc."
88,Jan 10 2023 10:14AM,254640,10,ISDIN Corporation
96,Jan 10 2023 10:04AM,254639,10,ISDIN Corporation
97,Jan 10 2023 10:03AM,254638,10,ISDIN Corporation
98,Jan 10 2023 9:53AM,254637,10,"Citieffe, Inc."
100,Jan 10 2023 9:44AM,254636,10,"CLINUVEL, Inc."
101,Jan 10 2023 9:43AM,254635,10,"CLINUVEL, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 10 2023 10:20AM,254643,20,"Exact-Rx, Inc.",,,10
1,Jan 10 2023 10:18AM,254641,10,"Methapharm, Inc.",,,15
2,Jan 10 2023 10:15AM,254632,10,ISDIN Corporation,,,59
3,Jan 10 2023 10:14AM,254642,10,"Methapharm, Inc.",,,4
4,Jan 10 2023 10:14AM,254640,10,ISDIN Corporation,,,8
5,Jan 10 2023 10:04AM,254639,10,ISDIN Corporation,,,1
6,Jan 10 2023 10:03AM,254638,10,ISDIN Corporation,,,1
7,Jan 10 2023 9:53AM,254637,10,"Citieffe, Inc.",,,2
8,Jan 10 2023 9:44AM,254636,10,"CLINUVEL, Inc.",,,1
9,Jan 10 2023 9:43AM,254635,10,"CLINUVEL, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 10 2023 10:20AM,254643,20,"Exact-Rx, Inc.",10,,
1,Jan 10 2023 10:18AM,254641,10,"Methapharm, Inc.",15,,
2,Jan 10 2023 10:15AM,254632,10,ISDIN Corporation,59,,
3,Jan 10 2023 10:14AM,254642,10,"Methapharm, Inc.",4,,
4,Jan 10 2023 10:14AM,254640,10,ISDIN Corporation,8,,
5,Jan 10 2023 10:04AM,254639,10,ISDIN Corporation,1,,
6,Jan 10 2023 10:03AM,254638,10,ISDIN Corporation,1,,
7,Jan 10 2023 9:53AM,254637,10,"Citieffe, Inc.",2,,
8,Jan 10 2023 9:44AM,254636,10,"CLINUVEL, Inc.",1,,
9,Jan 10 2023 9:43AM,254635,10,"CLINUVEL, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 10 2023 10:20AM,254643,20,"Exact-Rx, Inc.",10,,
1,Jan 10 2023 10:18AM,254641,10,"Methapharm, Inc.",15,,
2,Jan 10 2023 10:15AM,254632,10,ISDIN Corporation,59,,
3,Jan 10 2023 10:14AM,254642,10,"Methapharm, Inc.",4,,
4,Jan 10 2023 10:14AM,254640,10,ISDIN Corporation,8,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 10 2023 10:20AM,254643,20,"Exact-Rx, Inc.",10.0,NaN,NaN
1,Jan 10 2023 10:18AM,254641,10,"Methapharm, Inc.",15.0,NaN,NaN
2,Jan 10 2023 10:15AM,254632,10,ISDIN Corporation,59.0,NaN,NaN
3,Jan 10 2023 10:14AM,254642,10,"Methapharm, Inc.",4.0,NaN,NaN
4,Jan 10 2023 10:14AM,254640,10,ISDIN Corporation,8.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 10 2023 10:20AM,254643,20,"Exact-Rx, Inc.",10.0,0.0,0.0
1,Jan 10 2023 10:18AM,254641,10,"Methapharm, Inc.",15.0,0.0,0.0
2,Jan 10 2023 10:15AM,254632,10,ISDIN Corporation,59.0,0.0,0.0
3,Jan 10 2023 10:14AM,254642,10,"Methapharm, Inc.",4.0,0.0,0.0
4,Jan 10 2023 10:14AM,254640,10,ISDIN Corporation,8.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2800949,94.0,8.0,3.0
15,2036800,76.0,7.0,37.0
16,254585,1.0,0.0,0.0
19,1018411,52.0,2.0,0.0
20,509268,11.0,0.0,0.0
21,763794,2.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2800949,94.0,8.0,3.0
1,15,2036800,76.0,7.0,37.0
2,16,254585,1.0,0.0,0.0
3,19,1018411,52.0,2.0,0.0
4,20,509268,11.0,0.0,0.0
5,21,763794,2.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,94.0,8.0,3.0
1,15,76.0,7.0,37.0
2,16,1.0,0.0,0.0
3,19,52.0,2.0,0.0
4,20,11.0,0.0,0.0
5,21,2.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,94.0
1,15,Released,76.0
2,16,Released,1.0
3,19,Released,52.0
4,20,Released,11.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21
Status,,,,,,
Completed,3.0,37.0,0.0,0.0,0.0,0.0
Executing,8.0,7.0,0.0,2.0,0.0,1.0
Released,94.0,76.0,1.0,52.0,11.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21
0,Completed,3.0,37.0,0.0,0.0,0.0,0.0
1,Executing,8.0,7.0,0.0,2.0,0.0,1.0
2,Released,94.0,76.0,1.0,52.0,11.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21
0,Completed,3.0,37.0,0.0,0.0,0.0,0.0
1,Executing,8.0,7.0,0.0,2.0,0.0,1.0
2,Released,94.0,76.0,1.0,52.0,11.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()